In [ ]:
import pandas as pd
import json

In [ ]:
fund_codes = ['MMM','ABT','ABBV']
startDate = '2020-01-31'
endDate = '2020-04-30'

In [ ]:
   response_columns = fund_codes
   response_columns.insert(0,'date')
   all_historical_prices = pd.read_csv('output.csv')

In [ ]:
subset_data = all_historical_prices[fund_codes][all_historical_prices['date']>=startDate ][all_historical_prices['date']<=endDate].sort_values('date').reset_index(drop = True)

In [ ]:
for i in fund_codes:
    subset_data[f'{i}_return'] = subset_data[i] / subset_data[i].shift(1) - 1
    subset_data = subset_data.fillna(0)

In [ ]:
subset_data = subset_data.fillna(0)
subset_data = subset_data.drop(fund_codes, axis = 1)
subset_data.columns = response_columns